# Model Metadata API Demo

The Model Metadata API is still in developement. This demo showcases some of the functionality of the new API.

## Api Configuration

In [1]:
from seldon_deploy_sdk.rest import ApiException

from seldon_deploy_sdk import V1Model, ModelMetadataServiceApi, Configuration, ApiClient
from seldon_deploy_sdk.auth import OIDCAuthenticator

In [2]:
from pprint import pprint
import json

In [3]:
config = Configuration()
config.host = "http://localhost:8080"

In [4]:
# auth if needed
# config.oidc_server = "http://localhost:8080/auth/realms/deploy-realm"
# config.oidc_client_id = "sd-cli"

# auth = OIDCAuthenticator(config)
# config.access_token = auth.authenticate("admin@seldon.io", "12341234")

In [5]:
api_client = ApiClient(config)
api_instance = ModelMetadataServiceApi(api_client)

## Add single model to the Model Catalogue

In [6]:
model = V1Model(
    uri="gs://test-model-alpha-v1.0.0",
    name="alpha",
    version="v1.0.0",
    implementation="XGBOOST",
    task_type="regression",
    tags={
        "source": "https://github.com/some-test-model-alpha-repo",
        "an arbitrary tag": "true",
    },
)
try:
    # Create a Model Metadata entry.
    api_response = api_instance.model_metadata_service_create_model_metadata(model)
except ApiException as e:
    print(f"Couldn't create model: {json.loads(e.body)['message']}")

## Add multiple models to the Model Catalogue

In [7]:
models = [
    #     Same model different versions
    {
        "uri": "gs://test-model-beta-v1.0.0",
        "name": "beta",
        "version": "v1.0.0",
        "implementation": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Jon"},
    },
    {
        "uri": "gs://test-model-beta-v2.0.0",
        "name": "beta",
        "version": "v2.0.0",
        "implementation": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Bob"},
    },
    {
        "uri": "gs://test-model-beta-v3.0.0",
        "name": "beta",
        "version": "v3.0.0",
        "implementation": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Bob"},
    },
]

for model in models:
    body = V1Model(**model)
    try:
        api_response = api_instance.model_metadata_service_create_model_metadata(body)
    except ApiException as e:
        print(f"Couldn't create model: {json.loads(e.body)['message']}")

## List all models in the Model Catalogue

In [8]:
try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata()
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'models': [{'implementation': 'sklearn',
             'name': 'kf-samples-iris',
             'tags': {'author': 'Harry Potter', 'team': 'Gryffindor'},
             'task_type': 'classification',
             'uri': 'gs://kfserving-samples/models/sklearn/iris',
             'version': 'v1.0.0'},
            {'implementation': 'sklearn',
             'name': 'seldon-models-iris',
             'tags': {'author': 'Ron Weasley', 'team': 'Gryffindor'},
             'task_type': 'classification',
             'uri': 'gs://seldon-models/sklearn/iris',
             'version': 'v1.0.0'},
            {'implementation': 'sklearn',
             'name': 'seldon-models-iris',
             'tags': {'author': 'Ron Weasley', 'team': 'Gryffindor'},
             'task_type': 'classification',
             'uri': 'gs://seldon-models/sklearn/iris2',
             'version': 'v2.0.0'},
            {'implementation': 'sklearn',
             'name': 'income',
             'tags': {'author': 'Open Source',
   

## Get all version of a given model (named "beta")

In [9]:
# uri = 'uri_example'                       # str |  (optional)
# name = 'name_example'                     # str |  (optional)
# version = 'version_example'               # str |  (optional)
# implementation = 'implementation_example' # str |  (optional)
# task_type = 'task_type_example'           # str |  (optional)
# model_type = 'model_type_example'         # str |  (optional)

try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(name="beta", tags={"author": "Jon"})
    print("Filter by name=beta")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

Filter by name=beta
{'models': [{'implementation': 'SKLEARN',
             'name': 'beta',
             'tags': {'author': 'Jon'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v1.0.0',
             'version': 'v1.0.0'}],
 'next_page_token': ''}


## Get all models authored by Bob (tags.author = Bob)

In [10]:
# uri = 'uri_example'                       # str |  (optional)
# name = 'name_example'                     # str |  (optional)
# version = 'version_example'               # str |  (optional)
# implementation = 'implementation_example' # str |  (optional)
# task_type = 'task_type_example'           # str |  (optional)
# model_type = 'model_type_example'         # str |  (optional)

try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(tags={"author": "Bob"})
    print("Filter by name=beta")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

Filter by name=beta
{'models': [{'implementation': 'SKLEARN',
             'name': 'beta',
             'tags': {'author': 'Bob'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v2.0.0',
             'version': 'v2.0.0'},
            {'implementation': 'SKLEARN',
             'name': 'beta',
             'tags': {'author': 'Bob'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v3.0.0',
             'version': 'v3.0.0'}],
 'next_page_token': ''}


## Modify model metadata entry in the Model Catalogue

In [11]:
try:
    # Get Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(uri="gs://test-model-alpha-v1.0.0")
    print("Before update:")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")


model = V1Model(
    uri="gs://test-model-alpha-v1.0.0",
    name="alpha",
    version="v1.0.0",
    implementation="XGBOOST",
    task_type="regression",
    tags={
        "source": "https://github.com/some-other-repo",
        "an arbitrary tag": "true",
        "an additional tag": "123",
    },
)

try:
    # Update a Model Metadata entry.
    api_response = api_instance.model_metadata_service_update_model_metadata(model)
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")
    
try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(
        uri="gs://test-model-alpha-v1.0.0"
    )
    print("After update:")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

Before update:
{'models': [{'implementation': 'XGBOOST',
             'name': 'alpha',
             'tags': {'an arbitrary tag': 'true',
                      'source': 'https://github.com/some-test-model-alpha-repo'},
             'task_type': 'regression',
             'uri': 'gs://test-model-alpha-v1.0.0',
             'version': 'v1.0.0'}],
 'next_page_token': ''}
{}
After update:
{'models': [{'implementation': 'XGBOOST',
             'name': 'alpha',
             'tags': {'an additional tag': '123',
                      'an arbitrary tag': 'true',
                      'source': 'https://github.com/some-other-repo'},
             'task_type': 'regression',
             'uri': 'gs://test-model-alpha-v1.0.0',
             'version': 'v1.0.0'}],
 'next_page_token': ''}


## Get runtime information for a model

In [12]:
try:
    # List Runtime Metadata for all deployments associated with a model.
    api_response = api_instance.model_metadata_service_list_runtime_metadata_for_model(
        model_uri="gs://kfserving-samples/models/sklearn/iris", 
        deployment_status="Running"
    )
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'next_page_token': '',
 'runtime_metadata': [{'deployment_kubernetes_uid': '235bd17d-7c17-4b24-9ab8-a2778413a9ac',
                       'deployment_name': 'iris',
                       'deployment_namespace': 'kfserving',
                       'deployment_status': 'Running',
                       'deployment_type': 'InferenceService',
                       'model_uri': 'gs://kfserving-samples/models/sklearn/iris',
                       'node_name': '',
                       'predictor_name': 'sklearn',
                       'shadow': False,
                       'traffic': '100'},
                      {'deployment_kubernetes_uid': '265bd17d-7c17-4b24-9ab8-a2778413a9ac',
                       'deployment_name': 'iris-2',
                       'deployment_namespace': 'kfserving',
                       'deployment_status': 'Running',
                       'deployment_type': 'InferenceService',
                       'model_uri': 'gs://kfserving-samples/models/sklearn/iris'

## Get model information for a deployment

In [13]:
try:
    # List Runtime Metadata for all deployments associated with a model.
    api_response = api_instance.model_metadata_service_list_runtime_metadata_for_model(
        deployment_name="iris", 
        deployment_namespace="seldon"
    )
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'next_page_token': '',
 'runtime_metadata': [{'deployment_kubernetes_uid': '123bd17d-7c17-4b24-9ab8-a2778413a9ac',
                       'deployment_name': 'iris',
                       'deployment_namespace': 'seldon',
                       'deployment_status': 'Running',
                       'deployment_type': 'SeldonDeployment',
                       'model_uri': 'gs://seldon-models/sklearn/iris',
                       'node_name': 'iris',
                       'predictor_name': 'default',
                       'shadow': False,
                       'traffic': '100'},
                      {'deployment_kubernetes_uid': '123bd17d-7c17-4b24-9ab8-a2778413a9ac',
                       'deployment_name': 'iris',
                       'deployment_namespace': 'seldon',
                       'deployment_status': 'Running',
                       'deployment_type': 'SeldonDeployment',
                       'model_uri': 'gs://seldon-models/sklearn/iris2',
                       